In [1]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from qdrant_client import QdrantClient
from qdrant_client import models
import numpy as np


client = QdrantClient(url="http://10.11.12.134:6333", timeout=20)
#client = QdrantClient("http://localhost", port=6333, grpc_port=6334)

COLLECTION_NAME = 'wikipedia_20231101_en_text512_embeddings'

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large',
    model_kwargs={"device": "cpu"},
    cache_folder='cache'
)

/tmp/ipykernel_200448/3294939613.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


In [3]:
import os
import json
import pandas as pd

DATA_DIR = '/data/semeval2025/v2/test'
# Чтение всех .jsonl файлов из директории
all_data = []
for file_name in os.listdir(DATA_DIR):
    if file_name.endswith('.jsonl'):
        file_path = os.path.join(DATA_DIR, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                all_data.append(json.loads(line))

# Построение DataFrame
df = pd.DataFrame(all_data)
df

,id,lang,model_input,model_output_text,model_id,model_output_tokens,model_output_logits
0,tst-en-1,EN,Did Alberto Fouillioux ever play in a world cu...,"No, Albero Foulois was not in any of the FIFA...",togethercomputer/Pythia-Chat-Base-7B,"[ĠNo, ,, ĠAlber, o, ĠF, oul, ois, Ġwas, Ġnot, ...","[-2.2868447304, 4.7311220169, 0.1059471965, 9...."
1,tst-en-2,EN,In which city was David Sandberg born?,"David Sandburg was born in Stockholm, Sweden.",tiiuae/falcon-7b-instruct,"[David, ĠSand, burg, Ġwas, Ġborn, Ġin, ĠStockh...","[-5.994849205, -14.9877281189, -11.5699691772,..."
2,tst-en-3,EN,When did Andrei Kravchuk begin working on The ...,Andreï Krivochkin started working in the Ital...,togethercomputer/Pythia-Chat-Base-7B,"[ĠAndre, Ã¯, ĠK, riv, och, kin, Ġstarted, Ġwor...","[-3.6730628014, 6.0303354263, 7.9428901672, -1..."
3,tst-en-4,EN,In which French department is Mouthier-Haute-P...,Mouthier is located in the department of Haute...,tiiuae/falcon-7b-instruct,"[M, outh, ier, Ġis, Ġlocated, Ġin, Ġthe, Ġdepa...","[-3.8519268036, -14.3682308197, -15.4611091614..."
4,tst-en-5,EN,Where was Paul Egede sent to do his missionary...,Paul Eggede was sent by the Dutch East India C...,tiiuae/falcon-7b-instruct,"[Paul, ĠE, gged, e, Ġwas, Ġsent, Ġby, Ġthe, ĠD...","[-6.1030082703, -12.9452104568, -10.9736461639..."
...,...,...,...,...,...,...,...
149,tst-en-150,EN,Who is the mother of the Aloadae?,"The mother was the goddess Aloê, who was also...",togethercomputer/Pythia-Chat-Base-7B,"[ĠThe, Ġmother, Ġwas, Ġthe, Ġgoddess, ĠAl, o, ...","[-4.5354504585, 6.6092157364, 4.6515207291, 5...."
150,tst-en-151,EN,When did filming for ‘’Hit Man' begin?,The film began filming in June 2016 in New Yo...,togethercomputer/Pythia-Chat-Base-7B,"[ĠThe, Ġfilm, Ġbegan, Ġfilming, Ġin, ĠJune, Ġ2...","[-5.2936048508, 3.7041149139, 10.3181667328, 5..."
151,tst-en-152,EN,Which family of false saber-toothed cats was n...,Copes' cat\n,togethercomputer/Pythia-Chat-Base-7B,"[ĠC, opes, ', Ġcat, Ċ]","[-4.7141532898, 4.5391974449, 2.1915411949, 0...."
152,tst-en-153,EN,What is the Garajonay National Park named after?,The park is named in honor of the nearby volc...,togethercomputer/Pythia-Chat-Base-7B,"[ĠThe, Ġpark, Ġis, Ġnamed, Ġin, Ġhonor, Ġof, Ġ...","[-4.1036319733, 3.6858282089, 10.1618499756, 9..."


In [4]:
# Добавление колонок doc1, ..., doc5
j = 0
#df = df[:10]
def fetch_top_docs(query):
    """Получает топ-5 документов из векторной БД."""
    global j
    embedding = embeddings.embed_query(f"query: {query}")
    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=embedding,
        limit=5,
        with_vectors=False,
        timeout=300
    )
    j += 1
    response = [
        {
            'id': result.id,
            'score': result.score,
            'text': result.payload['text'],
            'title': result.payload['my_metadata']['title'],
            'url': result.payload['my_metadata']['source']
        }
        for result in results.points
    ]
    print(j, len(results.points))
    return response  # Предполагается, что ID документов возвращаются в поле 'id'

# Добавляем новые колонки в DataFrame
for i in range(1, 6):
    df[f'doc{i}'] = df['model_input'].apply(lambda x: fetch_top_docs(x)[i-1])

# Сохраняем результат (если нужно)
output_path = '/data/semeval2025/semeval_test_RAG.json'
df.to_json(output_path)

1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
10 5
11 5
12 5
13 5
14 5
15 5
16 5
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
30 5
31 5
32 5
33 5
34 5
35 5
36 5
37 5
38 5
39 5
40 5
41 5
42 5
43 5
44 5
45 5
46 5
47 5
48 5
49 5
50 5
51 5
52 5
53 5
54 5
55 5
56 5
57 5
58 5
59 5
60 5
61 5
62 5
63 5
64 5
65 5
66 5
67 5
68 5
69 5
70 5
71 5
72 5
73 5
74 5
75 5
76 5
77 5
78 5
79 5
80 5
81 5
82 5
83 5
84 5
85 5
86 5
87 5
88 5
89 5
90 5
91 5
92 5
93 5
94 5
95 5
96 5
97 5
98 5
99 5
100 5
101 5
102 5
103 5
104 5
105 5
106 5
107 5
108 5
109 5
110 5
111 5
112 5
113 5
114 5
115 5
116 5
117 5
118 5
119 5
120 5
121 5
122 5
123 5
124 5
125 5
126 5
127 5
128 5
129 5
130 5
131 5
132 5
133 5
134 5
135 5
136 5
137 5
138 5
139 5
140 5
141 5
142 5
143 5
144 5
145 5
146 5
147 5
148 5
149 5
150 5
151 5
152 5
153 5
154 5
155 5
156 5
157 5
158 5
159 5
160 5
161 5
162 5
163 5
164 5
165 5
166 5
167 5
168 5
169 5
170 5
171 5
172 5
173 5
174 5
175 5
176 5
177 5
178 5
179 5
180 5
181 5
182 5
183 5
184 5
185 

In [26]:
df = pd.read_json('/data/semeval2025/semeval_val_RAG.json')

In [5]:
df

,id,lang,model_input,model_output_text,model_id,model_output_tokens,model_output_logits,doc1,doc2,doc3,doc4,doc5
0,tst-en-1,EN,Did Alberto Fouillioux ever play in a world cu...,"No, Albero Foulois was not in any of the FIFA...",togethercomputer/Pythia-Chat-Base-7B,"[ĠNo, ,, ĠAlber, o, ĠF, oul, ois, Ġwas, Ġnot, ...","[-2.2868447304, 4.7311220169, 0.1059471965, 9....","{'id': 10016768237, 'score': 0.87286377, 'text...","{'id': 10031114858, 'score': 0.8353081, 'text'...","{'id': 10041986913, 'score': 0.8336792, 'text'...","{'id': 10008821389, 'score': 0.8328247, 'text'...","{'id': 10055490096, 'score': 0.8327675, 'text'..."
1,tst-en-2,EN,In which city was David Sandberg born?,"David Sandburg was born in Stockholm, Sweden.",tiiuae/falcon-7b-instruct,"[David, ĠSand, burg, Ġwas, Ġborn, Ġin, ĠStockh...","[-5.994849205, -14.9877281189, -11.5699691772,...","{'id': 10051157738, 'score': 0.8515663, 'text'...","{'id': 10067081479, 'score': 0.8496475, 'text'...","{'id': 10004322723, 'score': 0.8480873, 'text'...","{'id': 10046591863, 'score': 0.8464966, 'text'...","{'id': 10069836755, 'score': 0.84236526, 'text..."
2,tst-en-3,EN,When did Andrei Kravchuk begin working on The ...,Andreï Krivochkin started working in the Ital...,togethercomputer/Pythia-Chat-Base-7B,"[ĠAndre, Ã¯, ĠK, riv, och, kin, Ġstarted, Ġwor...","[-3.6730628014, 6.0303354263, 7.9428901672, -1...","{'id': 10016507846, 'score': 0.8912277, 'text'...","{'id': 10010811095, 'score': 0.875309, 'text':...","{'id': 10002456055, 'score': 0.8068123, 'text'...","{'id': 10014287943, 'score': 0.80125046, 'text...","{'id': 10004538164, 'score': 0.7986202, 'text'..."
3,tst-en-4,EN,In which French department is Mouthier-Haute-P...,Mouthier is located in the department of Haute...,tiiuae/falcon-7b-instruct,"[M, outh, ier, Ġis, Ġlocated, Ġin, Ġthe, Ġdepa...","[-3.8519268036, -14.3682308197, -15.4611091614...","{'id': 10015594208, 'score': 0.91620255, 'text...","{'id': 10015927237, 'score': 0.86985207, 'text...","{'id': 10015736091, 'score': 0.86889076, 'text...","{'id': 10003273033, 'score': 0.86384964, 'text...","{'id': 10015934572, 'score': 0.86310005, 'text..."
4,tst-en-5,EN,Where was Paul Egede sent to do his missionary...,Paul Eggede was sent by the Dutch East India C...,tiiuae/falcon-7b-instruct,"[Paul, ĠE, gged, e, Ġwas, Ġsent, Ġby, Ġthe, ĠD...","[-6.1030082703, -12.9452104568, -10.9736461639...","{'id': 10001791119, 'score': 0.8878746, 'text'...","{'id': 10000318593, 'score': 0.8659859, 'text'...","{'id': 10049943393, 'score': 0.84836197, 'text...","{'id': 10026257714, 'score': 0.84555435, 'text...","{'id': 10028450364, 'score': 0.8441162, 'text'..."
...,...,...,...,...,...,...,...,...,...,...,...,...
149,tst-en-150,EN,Who is the mother of the Aloadae?,"The mother was the goddess Aloê, who was also...",togethercomputer/Pythia-Chat-Base-7B,"[ĠThe, Ġmother, Ġwas, Ġthe, Ġgoddess, ĠAl, o, ...","[-4.5354504585, 6.6092157364, 4.6515207291, 5....","{'id': 10000078535, 'score': 0.8710251, 'text'...","{'id': 10072645117, 'score': 0.8431587, 'text'...","{'id': 10028097536, 'score': 0.83179474, 'text...","{'id': 10045288446, 'score': 0.83088684, 'text...","{'id': 10037874543, 'score': 0.83086395, 'text..."
150,tst-en-151,EN,When did filming for ‘’Hit Man' begin?,The film began filming in June 2016 in New Yo...,togethercomputer/Pythia-Chat-Base-7B,"[ĠThe, Ġfilm, Ġbegan, Ġfilming, Ġin, ĠJune, Ġ2...","[-5.2936048508, 3.7041149139, 10.3181667328, 5...","{'id': 10070759642, 'score': 0.83493423, 'text...","{'id': 10006622565, 'score': 0.8283081, 'text'...","{'id': 10044001911, 'score': 0.8203354, 'text'...","{'id': 10026743188, 'score': 0.81266403, 'text...","{'id': 10002758912, 'score': 0.8107872, 'text'..."
151,tst-en-152,EN,Which family of false saber-toothed cats was n...,Copes' cat\n,togethercomputer/Pythia-Chat-Base-7B,"[ĠC, opes, ', Ġcat, Ċ]","[-4.7141532898, 4.5391974449, 2.1915411949, 0....","{'id': 10058521988, 'score': 0.8752327, 'text'...","{'id': 10000415000, 'score': 0.85889626, 'text.

In [9]:
df.loc[0]['doc1']

{'id': 10016768237,
 'score': 0.87286377,
 'text': "Alberto Jorge Fouillioux Ahumada  (22 November 1940 – 23 June 2018) was a Chilean football midfielder and striker who earned 70 caps and scored 12 goals for the Chile national team during his career.\n\nCareer\nFouillioux made his debut for Universidad Católica in 1957. He was part of two championship winning sides in 1961 and 1966. He played for Chile in two World Cups; the 1962 and 1966.\n\nIn 1969, he joined Huachipato and in 1972 he played for Unión Española. In the latter part of his career he played for Lille in France.\n\nAfter retiring as a player Fouillioux went on to become a manager, serving as head coach of Huachipato and Colo-Colo.\n\nFouillioux died on June 23, 2018, aged 77 in his birth city of Santiago, Chile.\n\nPersonal life\nFrom 1989 to 2000, Fouillioux worked as a football commentator and analyst for the Chilean TV channel Canal 13, making a renowned pair alongside Néstor Isella in the TV program . In addition, he